In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import networkx as nx
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

### Summarizing Text
Let's try out extractive summarization using the first four paragraphs of The Great Gatsby.

First, we'll try to extract the most representative sentence. Then, we'll extract keywords.

### Sentence extraction
The steps of our sentence extraction process:

Parse and tokenize the text using spaCy, and divide into sentences.
Calculate the tf-idf matrix.
Calculate similarity scores.
Calculate TextRank: We're going to use the ´networkx´ package to run the TextRank algorithm.
Let's get started!

In [2]:
# Importing the text the lazy way.
gatsby="In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. \"Whenever you feel like criticizing any one,\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\" He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intimate revelation was quivering on the horizon--for the intimate revelations of young men or at least the terms in which they express them are usually plagiaristic and marred by obvious suppressions. Reserving judgments is a matter of infinite hope. I am still a little afraid of missing something if I forget that, as my father snobbishly suggested, and I snobbishly repeat a sense of the fundamental decencies is parcelled out unequally at birth. And, after boasting this way of my tolerance, I come to the admission that it has a limit. Conduct may be founded on the hard rock or the wet marshes but after a certain point I don't care what it's founded on. When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart. Only Gatsby, the man who gives his name to this book, was exempt from my reaction--Gatsby who represented everything for which I have an unaffected scorn. If personality is an unbroken series of successful gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles away. This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the \"creative temperament\"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again. No--Gatsby turned out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams that temporarily closed out my interest in the abortive sorrows and short-winded elations of men."

# We want to use the standard english-language parser.
parser = spacy.load('en')

# Parsing Gatsby.
gatsby = parser(gatsby)

# Dividing the text into sentences and storing them as a list of strings.
sentences=[]
for span in gatsby.sents:
    # go from the start to the end of each span, returning each token in the sentence
    # combine each token using join()
    sent = ''.join(gatsby[i].string for i in range(span.start, span.end)).strip()
    sentences.append(sent)

# Creating the tf-idf matrix.
counter = TfidfVectorizer(lowercase=False, 
                          stop_words=None,
                          ngram_range=(1, 1), 
                          analyzer=u'word', 
                          max_df=.5, 
                          min_df=1,
                          max_features=None, 
                          vocabulary=None, 
                          binary=False)

#Applying the vectorizer
data_counts=counter.fit_transform(sentences)

### Similarity
So far, this is all (hopefully) familiar: We've done text parsing and the tf-idf calculation before. We should now have sentences represented as vectors, with each word having a score based on how often it occurs in the sentence divided by how often it occurs in the whole text.

Now let's calculate the similarity scores for the sentences and apply the TextRank algorithm. Because TextRank is based on Google's PageRank algorithm, the function is called 'pagerank'. The hyperparameters are the damping parameter ´alpha´ and the convergence parameter ´tol´.

In [3]:
# Calculating similarity
similarity = data_counts * data_counts.T

# Identifying the sentence with the highest rank.
nx_graph = nx.from_scipy_sparse_matrix(similarity)
ranks=nx.pagerank(nx_graph, alpha=.85, tol=.00000001)

ranked = sorted(((ranks[i],s) for i,s in enumerate(sentences)),
                reverse=True)
print(ranked[0])


(0.07458830063813306, 'This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the "creative temperament"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again.')


Since a lot of Gatsby is about the narrator acting as the observer of other peoples' sordid secrets, this seems pretty good. Now, let's extract some keywords.

### Keyword summarization
1) Parse and tokenize text (already done).
2) Filter out stopwords, choose only nouns and adjectives.
3) Calculate the neighbors of words (we'll use a window of 4).
4) Run TextRank on the neighbor matrix.

In [4]:
# Removing stop words and punctuation, then getting a list of all unique words in the text
gatsby_filt = [word for word in gatsby if word.is_stop==False and (word.pos_=='NOUN' or word.pos_=='ADJ')]
words=set(gatsby_filt)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency=pd.DataFrame(columns=words,index=words,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(gatsby):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in gatsby_filt]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(gatsby)-(len(gatsby)-(i+5)))
        # The potential neighbors.
        nextwords=gatsby[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby_filt for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1

print('done!')

done!


In [5]:
# Running TextRank
nx_words = nx.from_numpy_matrix(adjacency.as_matrix())
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(words)),
                reverse=True)
print(ranked[:5])

[(0.013842411165984558, hope), (0.012538179113556777, promises), (0.012538179113556777, exempt), (0.012455008769377494, glimpses), (0.012201713657423653, intimate)]


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


These results are less impressive. 'Hope', 'promises', and 'glimpses' certainly fit the elegiac, on-the-outside-looking-in tone of the book, but 'exempt' and 'world' are pretty generic. TextRank may perform better on a larger text sample.

### Drill
It is also possible that keyword phrases will work better. Modify the keyword extraction code to extract two-word phrases (digrams) rather than single words. Then try it with trigrams. You will probably want to broaden the window that defines 'neighbors.' Try a few different modifications, and write up your observations in your notebook. Discuss with your mentor.

A Tfid Vectorizer would work nicely in this situation.  Let's work through setting up a Tfid Vectorizer in order to extra the two-word phrases(digrams).

In [6]:
digram_extractor = TfidfVectorizer(lowercase=False, 
                                  stop_words=None,
                                  ngram_range=(2, 2), 
                                  analyzer=u'word', 
                                  max_df=.5, 
                                  min_df=1,
                                  max_features=None, 
                                  vocabulary=None, 
                                  binary=False)

#Applying the vectorizer
digram = digram_extractor.fit_transform(sentences)

In [8]:
print(digram_extractor.get_feature_names())

['And after', 'Conduct may', 'East last', 'Gatsby the', 'Gatsby turned', 'Gatsby what', 'Gatsby who', 'He didn', 'If personality', 'In consequence', 'In my', 'Most of', 'No Gatsby', 'Only Gatsby', 'Reserving judgments', 'The abnormal', 'This responsiveness', 'When came', 'Whenever you', 'abnormal mind', 'abortive sorrows', 'about him', 'about that', 'accused of', 'admission that', 'advantages that', 'advice that', 'afraid of', 'after boasting', 'after certain', 'all judgments', 'all right', 'all the', 'also made', 'always been', 'am still', 'an extraordinary', 'an intimate', 'an unaffected', 'an unbroken', 'and also', 'and at', 'and attach', 'and marred', 'and more', 'and short', 'and snobbishly', 'and so', 'and understood', 'and which', 'any more', 'any one', 'any other', 'appears in', 'are usually', 'as have', 'as if', 'as my', 'at birth', 'at least', 'at sort', 'at the', 'attach itself', 'attention forever', 'autumn felt', 'back from', 'be founded', 'be in', 'because was', 'been tur

In [9]:
gatsby_filt = [word.string.strip() for word in gatsby if word.is_stop==False and (word.pos_=='NOUN' or word.pos_=='ADJ')]

In [10]:
gatsby_filt

['younger',
 'vulnerable',
 'years',
 'father',
 'advice',
 'mind',
 'people',
 'world',
 'advantages',
 'communicative',
 'reserved',
 'way',
 'great',
 'deal',
 'consequence',
 'inclined',
 'judgments',
 'habit',
 'curious',
 'natures',
 'victim',
 'veteran',
 'bores',
 'abnormal',
 'mind',
 'quick',
 'quality',
 'normal',
 'person',
 'college',
 'politician',
 'privy',
 'secret',
 'wild',
 'unknown',
 'men',
 'confidences',
 'unsought',
 'sleep',
 'preoccupation',
 'hostile',
 'levity',
 'unmistakable',
 'sign',
 'intimate',
 'revelation',
 'horizon',
 'intimate',
 'revelations',
 'young',
 'men',
 'terms',
 'plagiaristic',
 'obvious',
 'suppressions',
 'judgments',
 'matter',
 'infinite',
 'hope',
 'little',
 'afraid',
 'father',
 'sense',
 'fundamental',
 'decencies',
 'birth',
 'way',
 'tolerance',
 'admission',
 'limit',
 'Conduct',
 'hard',
 'rock',
 'wet',
 'marshes',
 'certain',
 'point',
 'autumn',
 'world',
 'uniform',
 'sort',
 'moral',
 'attention',
 'riotous',
 'excursio

In [11]:
digrams = list(zip(gatsby_filt, gatsby_filt[1:]))
trigrams = list(zip(gatsby_filt, gatsby_filt[1:], gatsby_filt[2:]))

In [12]:
print(digrams)

[('younger', 'vulnerable'), ('vulnerable', 'years'), ('years', 'father'), ('father', 'advice'), ('advice', 'mind'), ('mind', 'people'), ('people', 'world'), ('world', 'advantages'), ('advantages', 'communicative'), ('communicative', 'reserved'), ('reserved', 'way'), ('way', 'great'), ('great', 'deal'), ('deal', 'consequence'), ('consequence', 'inclined'), ('inclined', 'judgments'), ('judgments', 'habit'), ('habit', 'curious'), ('curious', 'natures'), ('natures', 'victim'), ('victim', 'veteran'), ('veteran', 'bores'), ('bores', 'abnormal'), ('abnormal', 'mind'), ('mind', 'quick'), ('quick', 'quality'), ('quality', 'normal'), ('normal', 'person'), ('person', 'college'), ('college', 'politician'), ('politician', 'privy'), ('privy', 'secret'), ('secret', 'wild'), ('wild', 'unknown'), ('unknown', 'men'), ('men', 'confidences'), ('confidences', 'unsought'), ('unsought', 'sleep'), ('sleep', 'preoccupation'), ('preoccupation', 'hostile'), ('hostile', 'levity'), ('levity', 'unmistakable'), ('un

In [13]:
digrams = [' '.join(tup) for tup  in digrams]
trigrams = [' '.join(tup) for tup in trigrams]

In [14]:
print(digrams)

['younger vulnerable', 'vulnerable years', 'years father', 'father advice', 'advice mind', 'mind people', 'people world', 'world advantages', 'advantages communicative', 'communicative reserved', 'reserved way', 'way great', 'great deal', 'deal consequence', 'consequence inclined', 'inclined judgments', 'judgments habit', 'habit curious', 'curious natures', 'natures victim', 'victim veteran', 'veteran bores', 'bores abnormal', 'abnormal mind', 'mind quick', 'quick quality', 'quality normal', 'normal person', 'person college', 'college politician', 'politician privy', 'privy secret', 'secret wild', 'wild unknown', 'unknown men', 'men confidences', 'confidences unsought', 'unsought sleep', 'sleep preoccupation', 'preoccupation hostile', 'hostile levity', 'levity unmistakable', 'unmistakable sign', 'sign intimate', 'intimate revelation', 'revelation horizon', 'horizon intimate', 'intimate revelations', 'revelations young', 'young men', 'men terms', 'terms plagiaristic', 'plagiaristic obvi

In [23]:
#Creating a grid indicating whether words are within 4 places of the target word
adjacency_digram=pd.DataFrame(columns=digrams,index=digrams,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(digrams):
   
    end=max(0,len(digrams)-(len(digrams)-(i+5)))
    # The potential neighbors.
    nextwords=digrams[i+1:end]
    # Filtering the neighbors to select only those in the word list
   
    neighbors=[nextwords[i] for i in range(len(nextwords))] 
    # Adding 1 to the adjacency matrix for neighbors of the target word
    if neighbors:
        adjacency_digram.loc[word,neighbors]=adjacency_digram.loc[word,neighbors]+1

print('done!')

done!


In [24]:
# Running TextRank

nx_words = nx.from_numpy_matrix(adjacency_digram.values)
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(digrams)),
                reverse=True)
print(ranked[:5])

[(0.008741685280630193, 'interest abortive'), (0.008741685280630192, 'advice mind'), (0.008549072337288351, 'mind people'), (0.00854907233728835, 'dreams interest'), (0.008396691872462302, 'people world')]


Try it with the trigram now

In [25]:
#Creating a grid indicating whether words are within 4 places of the target word
adjacency_trigram=pd.DataFrame(columns=trigrams,index=trigrams,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(trigrams):
   
    end=max(0,len(trigrams)-(len(trigrams)-(i+5)))
    # The potential neighbors.
    nextwords=trigrams[i+1:end]
    # Filtering the neighbors to select only those in the word list
   
    neighbors=[nextwords[i] for i in range(len(nextwords))] 
    # Adding 1 to the adjacency matrix for neighbors of the target word
    if neighbors:
        adjacency_trigram.loc[word,neighbors]=adjacency_trigram.loc[word,neighbors]+1

print('done!')

done!


In [26]:
# Running TextRank

nx_words = nx.from_numpy_matrix(adjacency_trigram.values)
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(trigrams)),
                reverse=True)
print(ranked[:5])

[(0.008809979696885118, 'dreams interest abortive'), (0.008809979696885118, 'advice mind people'), (0.008615861964923418, 'mind people world'), (0.008615861964923416, 'wake dreams interest'), (0.008462291027715913, 'people world advantages')]
